https://github.com/MIT-LCP/wfdb-python/blob/master/demo.ipynb

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import shutil
import wfdb

In [ ]:
# Demo 1 - Read a wfdb record using the 'rdrecord' function into a wfdb.Record object.
# Plot the signals, and show the data.
record = wfdb.rdrecord('./wfdb-python/sample-data/a103l')
# rdrecordを使ってwfdbレコードを読む

In [ ]:
wfdb.plot_wfdb(record = record, title = "Record a103l from Physionet Challenge 2015")
#プロット

In [ ]:
display(record.__dict__)
# メタデータを見る

In [ ]:
# Demo 2 - Read certain channels and sections of the WFDB record using the simplified 'rdsamp' function
# which returns a numpy array and a dictionary. Show the data.

signals, fields = wfdb.rdsamp('./wfdb-python/sample-data/s0010_re', channels=[14, 0, 5, 10], sampfrom=100, sampto=15000)
# channelがなんでこの形なのかワカラン→　List of integer indices specifying the channels to be read. Leave as ‘all’ to read all channels.
# つまり、channelの[14番、0番、5番、10番]の順に並んでいる？？


In [ ]:
display(signals)

In [ ]:
display(fields)

In [ ]:
# Demo 3 - Read a WFDB header file only (without the signals)
record = wfdb.rdheader('./wfdb-python/sample-data/drive02')
display(record.__dict__)


In [ ]:
# Demo 4 - Read part of a WFDB annotation file into a wfdb.Annotation object, and plot the samples
annotation = wfdb.rdann('./wfdb-python/sample-data/100', 'atr', sampfrom=100000, sampto=110000)
annotation.fs = 360
wfdb.plot_wfdb(annotation=annotation, time_units='minutes')

In [ ]:
# Demo 5 - Read a WFDB record and annotation. Plot all channels, and the annotation on top of channel 0.
record = wfdb.rdrecord('./wfdb-python/sample-data/100', sampto = 15000)
annotation = wfdb.rdann('./wfdb-python/sample-data/100', 'atr', sampto = 15000)

In [ ]:
wfdb.plot_wfdb(record=record, annotation=annotation,
               title='Record 100 from MIT-BIH Arrhythmia Database',
               time_units='seconds')

In [ ]:
# Demo 6 - Read the multi-segment record and plot waveforms from the MIMIC matched waveform database. 
record = wfdb.rdrecord('./wfdb-python/sample-data/multi-segment/s25047/s25047-2704-05-04-10-44')

In [ ]:
wfdb.plot_wfdb(record=record, title='Record s25047-2704-05-04-10-44') 

In [ ]:
display(record.__dict__)

In [ ]:
# Demo 7 - Read the multi-segment record and plot waveforms from the MIMIC matched waveform database.
# Notice that some channels have no valid values to plot
record = wfdb.rdrecord('./wfdb-python/sample-data/multi-segment/s00001/s00001-2896-10-10-00-31',
                       sampfrom=3000000, sampto=4000000)

In [ ]:
wfdb.plot_wfdb(record, title='Record s00001/s00001-2896-10-10-00-31') 

In [ ]:
display(record.__dict__)

In [ ]:
# Demo 8 - Read a wfdb record in which one channel has multiple samples/frame. Return a smoothed uniform array.
record = wfdb.rdrecord('./wfdb-python/sample-data/test01_00s_frame')

In [ ]:
wfdb.plot_wfdb(record)

In [ ]:
# Demo 9 - Read a wfdb record in which one channel has multiple samples/frame. Return a list of all the expanded samples.
record = wfdb.rdrecord('./wfdb-python/sample-data/test01_00s_frame', smooth_frames=False)

In [ ]:
display(record.e_p_signal)

In [ ]:
# Show that different channels have different lengths. Channel 1 has 2 samples/frame, hence has 2x as many samples.
print([len(s) for s in record.e_p_signal])

In [ ]:
import wfdb
from wfdb import processing

In [ ]:
# Demo 19 - Use the gqrs detection algorithm and correct the peaks
def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()

In [ ]:
# Load the wfdb record and the physical samples
record = wfdb.rdrecord('./wfdb-python/sample-data/100', sampfrom=0, sampto=10000, channels=[0])

# Use the gqrs algorithm to detect qrs locations in the first channel
qrs_inds = processing.gqrs_detect(sig=record.p_signal[:,0], fs=record.fs)

In [ ]:
# Plot results
peaks_hr(sig=record.p_signal, peak_inds=qrs_inds, fs=record.fs,
        title="GQRS peak detection on record 100")

In [ ]:
# Correct the peaks shifting them to local maxima
min_bpm = 20
max_bpm = 230
#min_gap = record.fs * 60 / min_bpm
# Use the maximum possible bpm as the search radius
search_radius = int(record.fs * 60 / max_bpm)
corrected_peak_inds = processing.correct_peaks(record.p_signal[:,0], peak_inds=qrs_inds,
                                               search_radius=search_radius, smooth_window_size=150)

In [ ]:
# Display results
print('Corrected gqrs detected peak indices:', sorted(corrected_peak_inds))
peaks_hr(sig=record.p_signal, peak_inds=sorted(corrected_peak_inds), fs=record.fs,
         title="Corrected GQRS peak detection on sampledata/100")

In [ ]:
# Demo 20 - Use the xqrs detection algorithm and compare results to reference annotations
sig, fields = wfdb.rdsamp('./wfdb-python/sample-data/100', channels=[0], sampto=15000)
ann_ref = wfdb.rdann('./wfdb-python/sample-data/100','atr', sampto=15000)

In [ ]:
# Run qrs detection on signal
xqrs = processing.XQRS(sig=sig[:,0], fs=fields['fs'])
xqrs.detect()
# Alternatively, use the gateway function to get the qrs indices directly
# qrs_inds = processing.xqrs_detect(sig=sig[:,0], fs=fields['fs'])

# Compare detected qrs complexes to reference annotation.
# Note, first sample in 100.atr is not a qrs.
comparitor = processing.compare_annotations(ref_sample=ann_ref.sample[1:],
                                            test_sample=xqrs.qrs_inds,
                                            window_width=int(0.1 * fields['fs']),
                                            signal=sig[:,0])


In [ ]:
# Print and plot the results
comparitor.print_summary()
comparitor.plot(title='xqrs detected qrs vs reference annotations')